In [23]:
import pandas as pd

# Load the data
id_table = pd.read_excel("./data/id_table.xlsx")
demographics = pd.read_excel("./data/demographics.xlsx", skiprows=1, header=0)
scores_xls = pd.ExcelFile("./data/score_20241008.xlsx")

# Define the function to generate fmri codes
def generate_fmri_code_demo(row):
    prefix = 's' if row['참여집단'] == 'EXP' else 'c'
    return f"{prefix}{int(row['내부 부여 번호']):04d}"

def generate_fmri_code_id_table(row):
    prefix = 's' if row['Enrollment #'].startswith('EXP') else 'c'
    code = f"{prefix}{int(row['Subject #']):04d}"
    return code

# Apply the function to generate fmri codes
demographics['fmri_code'] = demographics.apply(generate_fmri_code_demo, axis=1)
id_table['fmri_code'] = id_table.apply(generate_fmri_code_id_table, axis=1)

# Merge the two DataFrames on fmri_code
merged_df = pd.merge(demographics, id_table, on=['fmri_code'], how='left')

# Drop rows where fmri_code is NaN
filtered_df = merged_df.dropna(subset=['fmri_code'])

# Convert 'HAID ID' to string for consistent merging
filtered_df['HAID ID'] = filtered_df['HAID ID'].astype(str)

# Load the score sheets
STAI_X_1 = scores_xls.parse("STAI_X_1")
STAI_X_2 = scores_xls.parse("STAI_X_2")
HADS = scores_xls.parse("HADS")
SWLS = scores_xls.parse("SWLS")
GAD_7 = scores_xls.parse("GAD_7")
PDSS = scores_xls.parse("PDSS")
LSAS = scores_xls.parse("LSAS")
MOCI = scores_xls.parse("MOCI")
BFNE = scores_xls.parse("BFNE")
PSWQ = scores_xls.parse("PSWQ")
FCV_19S = scores_xls.parse("FCV_19S")
HANDEDNESS = scores_xls.parse("HANDEDNESS")

# Create LSAS new variables
exclude_columns = ['performance_lsas', 'social_interaction_lsas', 'lsas', 'round', 'id', 'date']
LSAS['LSAS_performance_fear'] = LSAS.drop(columns=exclude_columns).filter(regex=r'^p\d+f$').sum(axis=1)
LSAS['LSAS_performance_avoidance'] = LSAS.drop(columns=exclude_columns).filter(regex=r'^p\d+a$').sum(axis=1)
LSAS['LSAS_social_fear'] = LSAS.drop(columns=exclude_columns).filter(regex=r'^s\d+f$').sum(axis=1)
LSAS['LSAS_social_avoidance'] = LSAS.drop(columns=exclude_columns).filter(regex=r'^s\d+a$').sum(axis=1)
LSAS['LSAS_fear'] = LSAS.drop(columns=exclude_columns).filter(regex=r'.*f$').sum(axis=1)
LSAS['LSAS_avoidance'] = LSAS.drop(columns=exclude_columns).filter(regex=r'.*a$').sum(axis=1)

# Extract scores function
def extract_scores(df, columns, id):
    extracted_col = df[(df['id'] == id) & (df['round'] == 1)][columns]
    if len(extracted_col.values) == 0:
        return "n/a"
    else:
        return extracted_col.values[0][0]

# Dictionary to hold the extracted scores for each id
extracted_scores = []

# Loop through each unique HAID ID and extract the scores for each sheet
for id in filtered_df['HAID ID'].unique():
    target_filtered_df = filtered_df[filtered_df['HAID ID'] == id]
    score_dict = {
        'HAID ID': id,
        'GROUP': target_filtered_df['참여집단'].values[0],
        'Exp No.': target_filtered_df['내부 부여 번호'].values[0],
        '1. SEX': target_filtered_df['성별'].values[0],
        '2.AGE': target_filtered_df['만 나이'].values[0],
        '3-2. YR_EDU': target_filtered_df['교육 기간'].values[0],
        'fmri_code': target_filtered_df['fmri_code'].values[0],
        'Screening #': target_filtered_df['Screening #'].values[0],
        'Enrollment #': target_filtered_df['Enrollment #'].values[0],
        
        'STAI-X-1': extract_scores(STAI_X_1, ['STAI-X-1'], id),
        'STAI-X-2': extract_scores(STAI_X_2, ['STAI-X-2'], id),
        'HADS_anxiety': extract_scores(HADS, ['HADS_anxiety'], id),
        'HADS_depression': extract_scores(HADS, ['HADS_depression'], id),
        'SWLS': extract_scores(SWLS, ['SWLS'], id),
        'GAD-7': extract_scores(GAD_7, ['GAD-7'], id),
        'PDSS': extract_scores(PDSS, ['PDSS'], id),
        'LSAS_performance': extract_scores(LSAS, ['performance_lsas'], id),
        'LSAS_social_interaction': extract_scores(LSAS, ['social_interaction_lsas'], id),
        'LSAS': extract_scores(LSAS, ['lsas'], id),
        'MOCI': extract_scores(MOCI, ['MOCI'], id),
        'MOCI_checking': extract_scores(MOCI, ['checking'], id),
        'MOCI_cleaning': extract_scores(MOCI, ['cleaning'], id),
        'MOCI_doubting': extract_scores(MOCI, ['doubting'], id),
        'MOCI_slowness': extract_scores(MOCI, ['slowness'], id),
        'BFNE': extract_scores(BFNE, ['BFNE'], id),
        'PSWQ': extract_scores(PSWQ, ['PSWQ'], id),
        'LSAS_performance_fear': extract_scores(LSAS, ['LSAS_performance_fear'], id),
        'LSAS_performance_avoidance': extract_scores(LSAS, ['LSAS_performance_avoidance'], id),
        'LSAS_social_fear': extract_scores(LSAS, ['LSAS_social_fear'], id),
        'LSAS_social_avoidance': extract_scores(LSAS, ['LSAS_social_avoidance'], id),
        'LSAS_fear': extract_scores(LSAS, ['LSAS_fear'], id),
        'LSAS_avoidance': extract_scores(LSAS, ['LSAS_avoidance'], id),
        'FCV-19S': extract_scores(FCV_19S, ['FCV-19S'], id)
    }
    extracted_scores.append(score_dict)

# Convert to DataFrame for further analysis
extracted_scores_df = pd.DataFrame(extracted_scores)


In [16]:
demographics.columns


Index(['참여집단', '내부 부여 번호', '병록번호', '이름', '성별', '만 나이', '학력', '교육 기간', '직업',
       '자신', '부모', '정신질환\n가족/친척 유무', '참가자와 관계', '정신과입원유무', '질환명', '현재 질병 유무',
       '병명', '과거 뇌외상/\n뇌질환', '정신질환 유무', '병명 ', '입원횟수', '첫 정신과적 입원\n(몇년 전)',
       '첫 정신과적 입원\n(입원 년도)', '발병시기\n(몇년 전)', '발병시기\n(입원 년도)', '항정신병 약물',
       '향정신병약물/ \n용량(mg/tab)', '용량\n(tab)', '현 용량 시작일', '기타 약물\n(약품명/용량)',
       '총 약물 치료기간', 'fmri_code'],
      dtype='object')

In [17]:
# Convert 'HAID ID' to string for consistent merging
filtered_df['HAID ID'] = filtered_df['HAID ID'].astype(str)

In [19]:
extracted_scores_df = pd.DataFrame(extracted_scores)

In [24]:
extracted_scores_df

,HAID ID,GROUP,Exp No.,1. SEX,2.AGE,3-2. YR_EDU,fmri_code,Screening #,Enrollment #,STAI-X-1,...,MOCI_slowness,BFNE,PSWQ,LSAS_performance_fear,LSAS_performance_avoidance,LSAS_social_fear,LSAS_social_avoidance,LSAS_fear,LSAS_avoidance,FCV-19S
0,0827jck,EXP,7,2,20,12.0,s0007,1.0,EXP-0001,56,...,3,56,65,29,22,25,22.0,54,44.0,14
1,na0840,EXP,5,2,20,12.0,s0005,2.0,EXP-0002,67,...,5,57,71,23,27,14,16.0,37,43.0,25
2,spwls915,EXP,4,2,21,15.0,s0004,3.0,EXP-0003,65,...,2,59,69,25,26,20,24.0,45,50.0,12
3,wkddbswns,HC,9,1,25,15.0,c0009,4.0,HC-0001,35,...,0,35,52,6,3,5,4.0,11,7.0,9
4,bje5409,EXP,12,2,30,12.0,s0012,5.0,EXP-0004,55,...,0,51,55,20,17,9,11.0,29,28.0,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
159,kimhg2004,EXP,574,2,19,12.0,s0574,160.0,EXP-0086,42,...,2,47,45,16,20,12,17.0,28,37.0,12
160,miss20c,EXP,562,2,39,20.0,s0562,161.0,EXP-0087,58,...,1,44,58,14,5,12,4.0,26,9.0,11
161,goun4238,EXP,583,1,19,12.0,s0583,162.0,EXP-0088,70,...,6,53,74,28,26,26,25.0,54,51.0,13
162,yyyyjg,HC,549,2,35,NaN,c0549,167.0,HC-0076,34,...,1,41,52,4,9,2,9.0,6,18.0,7


In [25]:
extracted_scores_df.to_csv("./data/participant_demo_clinical_all_new.csv")